In [ ]:
%%capture
!git clone https://github.com/adelic99/seeker.git
%cd seeker

# Upgrade pip
!python -m pip install -U pip

!pip install -U numpy scipy pandas tqdm pyyaml matplotlib scikit-learn einops
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
!ls /kaggle/input/stg-nf/data/data/ShanghaiTech/pose | head

test
train


In [3]:
!find /kaggle/input/stg-nf/data/data/ShanghaiTech/pose -name "*tracked_person.json" | head

/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/07_0005_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/01_0064_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/04_0012_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/01_0133_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/10_0037_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/06_0150_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/01_0016_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/12_0174_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/08_0080_tracked_person.json
/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test/08_0179_tracked_person.json
find: ‘standard output’: Broken pipe
find: write error


In [4]:
import glob, os

train_dir = "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/train"
test_dir  = "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test"

print("train tracked_person.json files:",
      len(glob.glob(os.path.join(train_dir, "*tracked_person.json"))))
print("test  tracked_person.json files:",
      len(glob.glob(os.path.join(test_dir, "*tracked_person.json"))))

train tracked_person.json files: 320
test  tracked_person.json files: 107


In [ ]:
# =========================
# Dataset
# =========================
dataset = "ShanghaiTech"

pose_path = {
    "train": "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/train",
    "test":  "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test",
}

vid_path = {
    "train": "/kaggle/input/stg-nf/data/data/ShanghaiTech",
    "test":  "/kaggle/input/stg-nf/data/data/ShanghaiTech",
}

# =========================
# Training params 
# =========================
batch_size = 64      # nếu OOM → 32
num_workers = 2
seg_len = 12
seg_stride = 1
filter_conf = 0
train_seg_conf_th = 0.0

In [6]:
# Fix validation GT path by creating the expected symlink:
# Repo expects:  data/ShanghaiTech/gt/test_frame_mask/
# Your GT is at: /kaggle/input/stg-nf/data/data/ShanghaiTech/gt/test_frame_mask

%cd /kaggle/working/seeker

!mkdir -p data/ShanghaiTech/gt

# Remove if a broken link already exists
!rm -rf data/ShanghaiTech/gt/test_frame_mask

# Create symlink
!ln -s "/kaggle/input/stg-nf/data/data/ShanghaiTech/gt/test_frame_mask" "data/ShanghaiTech/gt/test_frame_mask"

# Verify
!ls -la data/ShanghaiTech/gt/test_frame_mask | head

/kaggle/working/seeker
lrwxrwxrwx 1 root root 62 Jan 28 13:39 data/ShanghaiTech/gt/test_frame_mask -> /kaggle/input/stg-nf/data/data/ShanghaiTech/gt/test_frame_mask


In [7]:
%cd /kaggle/working/seeker

!python - <<'PY'
from pathlib import Path
p = Path("seeker.py")
s = p.read_text(encoding="utf-8")
s2 = s.replace("args.dropout", "args.droppout")
if s2 == s:
    raise SystemExit("Không tìm thấy 'args.dropout' để thay. Hãy kiểm tra đường dẫn file seeker.py.")
p.write_text(s2, encoding="utf-8")
print("Patched: args.dropout -> args.droppout")

/kaggle/working/seeker
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
Patched: args.dropout -> args.droppout


In [8]:
%cd /kaggle/working/seeker

!python - <<'PY'
from pathlib import Path
p = Path("training.py")
s = p.read_text(encoding="utf-8")

# Replace first occurrence of torch.load(filename) -> torch.load(filename, weights_only=False)
if "torch.load(filename)" in s and "weights_only" not in s:
    s = s.replace("torch.load(filename)", "torch.load(filename, weights_only=False)", 1)
    p.write_text(s, encoding="utf-8")
    print("Patched training.py: torch.load(..., weights_only=False)")
else:
    print("No patch applied (pattern not found or already patched).")

/kaggle/working/seeker
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
Patched training.py: torch.load(..., weights_only=False)


In [9]:
%cd /kaggle/working/seeker

!python - <<'PY'
from pathlib import Path
import re

p = Path("training.py")
s = p.read_text(encoding="utf-8")

# Replace the line that unpacks 3 values into a robust block
pattern = r"auc,\s*frame_scores,\s*frame_gt\s*=\s*score_anomalies\([^\n]*\)\n"
m = re.search(pattern, s)
if not m:
    raise SystemExit("ERROR: Could not find the unpacking line in training.py. Search manually for 'auc, frame_scores, frame_gt'.")

replacement = (
    "ret = score_anomalies(anomaly_scores_kp, self.test_metadata, args=self.args, split='test', sigma=sigma)\n"
    "        if isinstance(ret, tuple) and len(ret) == 3:\n"
    "            auc, frame_scores, frame_gt = ret\n"
    "        else:\n"
    "            auc = ret\n"
    "            frame_scores, frame_gt = None, None\n"
)
# keep same indentation as the original line
indent = re.match(r"(\s*)auc", m.group(0)).group(1)
replacement = "".join(indent + line if line.strip() else line for line in replacement.splitlines(True))

s2 = s[:m.start()] + replacement + s[m.end():]
p.write_text(s2, encoding="utf-8")
print("Patched training.py: test() now handles score_anomalies returning float or (auc, scores, gt)")

/kaggle/working/seeker
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
Patched training.py: test() now handles score_anomalies returning float or (auc, scores, gt)


In [17]:
!python seeker.py \
  --dataset ShanghaiTech \
  --data_dir "/kaggle/input/stg-nf/data/data" \
  --pose_path_test "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test" \
  --checkpoint "/kaggle/working/seeker/exp_dir/ShanghaiTech/Jan28_0810/checkpoint_best.pth" \
  --exp_dir "/kaggle/working/seeker_eval_sigma10" \
  --seg_len 12 --seg_stride 1 --droppout 0.1 \
  --batch_size 64 --num_workers 4 \
  --sigma 10

2026-01-28 10:09:45.914910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769594985.936210     947 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769594985.943007     947 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769594985.959624     947 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769594985.959657     947 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769594985.959661     947 computation_placer.cc:177] computation placer alr

In [16]:
!python - <<'PY'
import torch
ckpt = torch.load("/kaggle/working/seeker/exp_dir/ShanghaiTech/Jan28_0810/checkpoint_best.pth", weights_only=False, map_location="cpu")
print("ckpt epoch:", ckpt.get("epoch"))
print("ckpt args:", ckpt.get("args"))

/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
ckpt epoch: 4
ckpt args: Namespace(vid_path_train=None, pose_path_train_abnormal=None, pose_path_train='/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/train', vid_path_validate=None, pose_path_validate=None, vid_path_test=None, pose_path_test='/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test', dataset='ShanghaiTech', device='cuda', seed=1692624671968321037, data_dir='/kaggle/input/stg-nf/data/data', exp_dir='/kaggle/working/seeker_runs', num_workers=2, droppout=0.1, train_seg_conf_th=0.0, seg_len=12, seg_stride=1, filter_conf=0.0, checkpoint=None, batch_size=64, sigma=0, epochs=30, model_optimizer='adam', model_lr=0.0005, n_layers=1, expansion_factor=1, vid_path={'train': '/kaggle/input/stg-nf/data/data/ShanghaiTech/train/images/', 'validate': '/kaggle/input/stg-nf/data/data/ShanghaiTech/validation/frames', 'test': '/kaggle/input/stg-nf/data/data/ShanghaiTech/test/frames/'}, pose_path={'tr

# CHẠY TRAIN

In [21]:
!python -u seeker.py \
  --dataset ShanghaiTech \
  --data_dir "/kaggle/input/stg-nf/data/data" \
  --pose_path_train "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/train" \
  --pose_path_test  "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test" \
  --seg_len 24 \
  --seg_stride 1 \
  --droppout 0.1 \
  --epochs 10 \
  --batch_size 128 \
  --model_lr 3e-4 \
  --sigma 10 \
  --num_workers 0 \
  --exp_dir "/kaggle/working/seeker_runs_paper_like"

2026-01-28 14:45:23.432274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769611523.615549     425 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769611523.670878     425 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769611524.128259     425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769611524.128306     425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769611524.128313     425 computation_placer.cc:177] computation placer alr

In [23]:
!python -u seeker.py \
  --dataset ShanghaiTech \
  --data_dir "/kaggle/input/stg-nf/data/data" \
  --pose_path_train "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/train" \
  --pose_path_test  "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test" \
  --seg_len 24 \
  --seg_stride 1 \
  --n_layers 2 \
  --expansion_factor 4 \
  --droppout 0.1 \
  --epochs 10 \
  --batch_size 64 \
  --model_lr 3e-4 \
  --sigma 10 \
  --num_workers 0 \
  --exp_dir "/kaggle/working/seeker_runs_paper_like_big"

2026-01-28 14:54:23.683452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769612063.882523     453 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769612063.943610     453 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769612064.424252     453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769612064.424294     453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769612064.424298     453 computation_placer.cc:177] computation placer alr

In [24]:
!find /kaggle/working -type f -name "checkpoint_best.pth" | sort

/kaggle/working/seeker/exp_dir/ShanghaiTech/Jan28_1406/checkpoint_best.pth


In [25]:
from pathlib import Path
import re

p = Path("/kaggle/working/seeker/training.py")
assert p.exists(), f"Không tìm thấy: {p}"

s = p.read_text(encoding="utf-8")

# Nếu đã có thì thôi
if "def joint_lnp_full_window" in s:
    print("OK: training.py đã có joint_lnp_full_window (không cần patch).")
else:
    # Chèn method ngay trước def test(self) trong class SeeKerTrainer
    m = re.search(r"\n(\s+def test\s*\(self[^\n]*\)\s*:\s*\n)", s)
    if not m:
        raise RuntimeError("Không tìm thấy 'def test(self' trong training.py để chèn patch.")

    indent = re.match(r"\n(\s+)def test", m.group(0)).group(1)

    wrapper = (
        f"\n{indent}def joint_lnp_full_window(self, x):\n"
        f"{indent}    \"\"\"Compatibility wrapper for public repo.\"\"\"\n"
        f"{indent}    if hasattr(self, 'joint_lnp'):\n"
        f"{indent}        return self.joint_lnp(x)\n"
        f"{indent}    # fallback\n"
        f"{indent}    return self.model(x)\n\n"
    )

    s2 = s[:m.start(1)] + wrapper + s[m.start(1):]
    p.write_text(s2, encoding="utf-8")
    print("Patched: đã chèn joint_lnp_full_window() vào training.py")

# Verify
s_check = p.read_text(encoding="utf-8")
print("Verify:", "def joint_lnp_full_window" in s_check)

OK: training.py đã có joint_lnp_full_window (không cần patch).
Verify: True


In [26]:
!python seeker.py \
  --dataset ShanghaiTech \
  --data_dir "/kaggle/input/stg-nf/data/data" \
  --pose_path_test "/kaggle/input/stg-nf/data/data/ShanghaiTech/pose/test" \
  --checkpoint "/kaggle/working/seeker/exp_dir/ShanghaiTech/Jan28_1406/checkpoint_best.pth" \
  --seg_len 24 \
  --seg_stride 1 \
  --droppout 0.1 \
  --sigma 10 \
  --batch_size 64 \
  --num_workers 4 \
  --exp_dir "/kaggle/working/seeker_eval_paper"

2026-01-28 15:04:42.777212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769612682.972328     474 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769612683.027078     474 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769612683.496718     474 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769612683.496762     474 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769612683.496767     474 computation_placer.cc:177] computation placer alr